### Imports

In [2]:
import gwpopulation
gwpopulation.set_backend('jax')
import pickle as pkl
from bilby.core.result import read_in_result

# load in population-error package
from population_error import error_statistics

### Load in injection set and posterior set

In [3]:
# dictionary of injections, containing arrays of shape (Ninj). Must contain 'prior' key and should also contain 'total_generated'
with open('/home/jack.heinzel/public_html/voronoi_population/gw_nonparameteric/GWTC3/data/injection_set.pkl', 'rb') as ff:
    injections = pkl.load(ff)
    
# dictionary of gw samples, containing arrays of shape (Nobs, NPE). Must contain 'prior' key
with open('/home/jack.heinzel/public_html/voronoi_population/gw_nonparameteric/GWTC3/data/posteriors.pkl', 'rb') as ff:
    event_posteriors = pkl.load(ff)

### Create gwpopulation model

In [4]:
model_list = [
    gwpopulation.models.mass.SinglePeakSmoothedMassDistribution(), 
    gwpopulation.models.spin.iid_spin,
    gwpopulation.models.redshift.PowerLawRedshift()
    ]

model_function = gwpopulation.experimental.jax.NonCachingModel(model_list)

### Load in hyperposterior

In [5]:
gwtc3_result = read_in_result('/home/jack.heinzel/public_html/GWTC-3-population/GWTC-3-population-data/analyses/PowerLawPeak/o1o2o3_mass_c_iid_mag_iid_tilt_powerlaw_redshift_result.json')
hyperposterior = gwtc3_result.posterior

### Calculate error statistics

In [6]:
statistics = error_statistics(
    model_function, 
    injections, 
    event_posteriors, 
    hyperposterior, 
    include_likelihood_correction=True, 
    conversion_function=gwpopulation.conversions.convert_to_beta_parameters
    )

Nobs not provided, assuming Nobs = 69


Computing single event covariance weights integrated over hyperposterior samples: 100%|██████████| 11469/11469 [00:46<00:00, 244.12it/s]
Computing selection covariance weights integrated over hyperposterior samples: 100%|██████████| 11469/11469 [00:31<00:00, 368.56it/s]
For each posterior sample, average single event covariance with another posterior sample: 100%|██████████| 11469/11469 [00:32<00:00, 353.30it/s]
For each posterior sample, average selection covariance with another posterior sample: 100%|██████████| 11469/11469 [00:31<00:00, 366.32it/s]



Your inference loses approximately 0.263 bits of information to Monte Carlo approximations.
Of the total information loss
 * 0.255 bits is from uncertainty in the posterior. Of this
    * 11.0% is from the single-event Monte Carlo integration
    * 89.0% is from the selection Monte Carlo integration
 * 0.00827 bits is from bias in the posterior. Of the total bias
    * 3.0% is from the single-event Monte Carlo integration
    * 101.3% is from the selection Monte Carlo integration
    * -4.3% is from correlations in the uncertainty of the single-event and selection MC integrals
